In [1]:
import pandas as pd
import random
from datetime import datetime
import time
from itertools import islice
from math import sin, cos, sqrt, atan2, radians

In [2]:
url = "/bigdata/jelicicna/output_datasets/rides_table_nextbike_2020-09.csv.gz"

In [3]:
df = pd.read_csv(url, sep=';', low_memory=False)

In [4]:
df.head()

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time,uid_begin,uid_end,standing_before_ride,...,number,place_type,rack_locks,special_racks,spot,terminal_type,city,country_code,company,timezone
0,50.915359,6.928819,50.934330,6.938283,2020-09-01 00:47:02,2020-09-01 01:02:02,15.0,35268434,35283100,47.0,...,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
1,50.934330,6.938283,50.948824,6.991045,2020-09-01 01:52:02,2020-09-01 02:19:02,27.0,35283100,35284237,50.0,...,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
2,50.948824,6.991045,50.953270,7.013817,2020-09-01 21:37:03,2020-09-03 03:22:02,1785.0,35284237,35395707,1158.0,...,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
3,50.953270,7.013817,50.937835,6.984989,2020-09-04 19:48:02,2020-09-04 20:12:03,24.0,35395707,35505514,2426.0,...,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
4,50.937835,6.984989,50.936953,6.974491,2020-09-04 21:13:02,2020-09-04 21:21:02,8.0,35505514,35510956,61.0,...,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin


In [5]:
# approximate radius of earth in km
R = 6373.0

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
new_df = pd.DataFrame(columns= ['lat_begin', 'lng_begin', 'lat_end', 'lng_end', 'ride_begin', 'ride_end', 'ride_time',
                                'distance', 'uid_begin', 'uid_end', 'standing_before_ride', 
                                'address', 'name', 'available_bikes', 'bike', 'bike_racks', 'bikes', 
                                'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
                                'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
                                'country_code', 'company', 'timezone'] )

In [8]:
for index, row in df.iterrows():
    lat_begin = radians(row['lat_begin'])
    lng_begin = radians(row['lng_begin'])
    lat_end = radians(row['lat_end'])
    lng_end = radians(row['lng_end'])
    
    distance_lat = lat_end - lat_begin
    distance_lng = lng_end - lng_begin
    
    a = sin(distance_lat / 2)**2 + cos(lat_begin) * cos(lat_end) * sin(distance_lng / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = round(R * c, 3)*1000
    
    new_df = new_df.append({'lat_begin' : row['lat_begin'], 'lng_begin' : row['lng_begin'],
                            'lat_end' : row['lat_end'], 'lng_end' : row['lng_end'],
                            'ride_begin' : row['ride_begin'], 'ride_end' : row['ride_end'],
                            'ride_time' : row['ride_time'],
                            'distance' : distance,
                            'uid_begin' : row['uid_begin'], 'uid_end' : row['uid_end'], 
                            'standing_before_ride' : row['standing_before_ride'], 
                            'name' : row['name'], 'address' : row['address'],
                            'bike' : row['bike'], 'bike_racks' : row['bike_racks'], 
                            'bikes' : row['bikes'], 'booked_bikes' : row['booked_bikes'], 
                            'free_racks' : row['free_racks'], 'free_special_racks' : row['free_special_racks'],
                            'maintenance' : row['maintenance'], 'number' : row['number'], 
                            'place_type' : row['place_type'], 'rack_locks' : row['rack_locks'],
                            'special_racks' : row['special_racks'], 'spot' : row['spot'], 
                            'terminal_type' : row['terminal_type'], 'city' : row['city'],
                            'country_code' : row['country_code'], 'company' : row['company'],
                            'timezone' : row['timezone'],
                            'available_bikes' : row['available_bikes']}, ignore_index=True)

In [9]:
new_df.to_csv('/bigdata/jelicicna/output_datasets/distances_table_nextbike_2020-09.csv', sep=';', index=False)

In [ ]:
new_df

In [ ]:
single_bike_data = new_df[new_df['uid_end']==0]
single_bike_data

In [23]:
drop_list = single_bike_data.index.to_list()

In [24]:
for i in drop_list:
    if i in new_df.index.values:
        new_df.drop(i, inplace=True)

In [25]:
new_df = new_df.reset_index()
new_df.drop(['index'], axis=1, inplace=True)

In [26]:
single_bike_data = new_df[new_df['name']=="BIKE 21885"]
single_bike_data

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time,distance,uid_begin,uid_end,standing_before_ride,address,name,available_bikes,bike,bike_racks,bikes,booked_bikes,free_racks,free_special_racks,maintenance,number,place_type,rack_locks,special_racks,spot,terminal_type,city,country_code,company,timezone
14217,50.942009,6.957792,50.932140,6.987941,2021-01-02 12:20:01,2021-01-02 12:34:02,14.0,2381.0,40568011,40633449,2180.0,NaN,BIKE 21885,814,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14218,50.932140,6.987941,50.931935,6.988841,2021-01-02 14:02:01,2021-01-02 14:11:01,9.0,67.0,40633449,40635350,88.0,NaN,BIKE 21885,796,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14219,50.931935,6.988841,50.936380,6.986277,2021-01-02 15:19:01,2021-01-02 15:24:01,5.0,526.0,40635350,40636970,68.0,NaN,BIKE 21885,785,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14220,50.936380,6.986277,50.930399,6.992621,2021-01-02 17:57:01,2021-01-02 18:17:02,20.0,800.0,40636970,40640621,153.0,NaN,BIKE 21885,783,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14221,50.930399,6.992621,50.936799,6.974782,2021-01-09 16:57:02,2021-01-09 17:09:02,12.0,1439.0,40640621,40780012,10000.0,NaN,BIKE 21885,787,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14222,50.936799,6.974782,50.930714,6.994199,2021-01-09 17:26:01,2021-01-09 17:47:01,21.0,1520.0,40780012,40780696,17.0,NaN,BIKE 21885,758,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14223,50.930714,6.994199,50.930127,7.003095,2021-01-10 00:58:02,2021-01-10 01:04:02,6.0,627.0,40780696,40787151,431.0,NaN,BIKE 21885,759,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14224,50.930127,7.003095,50.950275,7.001366,2021-01-11 17:29:01,2021-01-11 17:44:01,15.0,2244.0,40787151,40822530,2425.0,NaN,BIKE 21885,776,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14225,50.950275,7.001366,50.953068,7.004696,2021-01-12 07:51:02,2021-01-12 07:58:01,7.0,388.0,40822530,40832735,847.0,NaN,BIKE 21885,747,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin
14226,50.953068,7.004696,50.953332,7.004415,2021-01-12 08:05:02,2021-01-12 08:06:01,1.0,35.0,40832735,40832903,7.0,NaN,BIKE 21885,792,True,0,1,0,0,0,False,0,12,False,0,False,NaN,Köln,DE,Nextbike,Europe/Berlin


In [ ]:
new_df.head(50)